# Drawing DEX price and candle charts

- This is an example notebook how to draw a price
candle chart for [Trading Strategy](https://tradingstrategy.ai).
- Please read [how to set up your environment first](./running).
- We draw an OHLCV chart. Internally `trading-strategy` is using
  [Plotly framework](https://plotly.com/python/candlestick-charts/#custom-candlestick-colors)
  for charting.

## Getting started

First, let's create Trading Strategy oracle market data client.
If you do not have an API key yet, you will be asked to create one.


In [16]:
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


## Resolve trading pair metadata

We look at **BNB / BUSD** trading pair on **PancakeSwap** DEX (version 2).

- To download the data we first need to download
  exchange and trading pair metadata,
  so that we can resolve symbolic exchange name and ticket
  information to the trading pair id
- We may have duplicate trading pair with the same ticket and token symbol information.
  This is often the case for popular trading pairs as there are a lot of scam
  pairs around with the same symbol. We go around this by using the option `pick_by_highest_vol`
  which gives us the correct trading pair.
- Note that the blockchain native token of EVM chain is presented by its wrapped version on DEXes.
  E.g. BNB on BNB Chain becomes WBNB.


In [17]:
from pyarrow import Table
from tradingstrategy.chain import ChainId
from tradingstrategy.exchange import ExchangeUniverse
from tradingstrategy.pair import PandasPairUniverse, DEXPair

# Fetch all exchange names, slugs and addresses
exchange_universe: ExchangeUniverse = client.fetch_exchange_universe()

# Fetch all trading pairs across all exchanges
pair_table: Table = client.fetch_pair_universe()
pair_universe = PandasPairUniverse(pair_table.to_pandas())

pancake_v2 = exchange_universe.get_by_chain_and_slug(ChainId.bsc, "pancakeswap-v2")

pair: DEXPair = pair_universe.get_one_pair_from_pandas_universe(
    pancake_v2.exchange_id,
    "WBNB",
    "BUSD",
    pick_by_highest_vol=True)

print(f"PancakeSwap v2 is {pancake_v2}")
print(f"We have trading pair {pair}")

PancakeSwap v2 is <Exchange PancakeSwap v2 at 0xca143ce32fe78f1f7019d7d551a6402fc5350c73 on Binance Smart Chain>
We have trading pair <Pair #1364760 WBNB - BUSD (0x58f876857a02d6762e0101bb5c46a8c1ed44dc16) at exchange #1187 on binance>


## Fetch price data feed

- We then downlaod OHLCV data using Trading Strategy oracle's real-time API endpoint
- The data is internally delivered as JSONL, but this is abstracted way by
  `trading-strategy` Python client library
- Any price of non-dollar pairs, e.g. AAVE/ETH, is pre-converted to US dollars
  by Trading Strategy exchange rate rules

In [18]:
import pandas as pd
from tradingstrategy.timebucket import TimeBucket

candles: pd.DataFrame = client.fetch_candles_by_pair_ids(
    {pair.pair_id},
    TimeBucket.d1,
    progress_bar_description=f"Download data for {pair.get_ticker()}"
)

## Inspect OHCLV data

- You can manually inspect OHCLV trading dataframes.
- Easiest way to do this is to use IPython `display()` function to render dataframes as table.


In [22]:
# Display latest 14 days of WBNB-BUSD data
display(candles.iloc[-14:])

,pair_id,timestamp,open,high,low,close,buy_volume,sell_volume,start_block,end_block,exchange_rate,buys,sells,avg,volume
timestamp,,,,,,,,,,,,,,,
2022-11-20,1364760,2022-11-20,272.413045,275.333610,262.266215,263.776560,6.221825e+06,7.596476e+06,23203540,23232072,1.0,20993.0,16248.0,NaN,1.381830e+07
2022-11-21,1364760,2022-11-21,263.782276,264.059049,246.742439,254.108152,1.825424e+07,1.979850e+07,23232073,23260489,1.0,20806.0,15551.0,NaN,3.805274e+07
2022-11-22,1364760,2022-11-22,254.106964,266.632127,252.086346,266.260659,9.225833e+06,7.205554e+06,23260491,23288794,1.0,18648.0,15903.0,NaN,1.643139e+07
2022-11-23,1364760,2022-11-23,266.261026,299.841244,264.315010,297.825673,2.729343e+07,2.228333e+07,23288796,23317233,1.0,21300.0,24550.0,NaN,4.957676e+07
2022-11-24,1364760,2022-11-24,297.822293,302.814980,292.549276,300.091218,9.670815e+06,9.369412e+06,23317237,23345600,1.0,16895.0,16145.0,NaN,1.904023e+07
2022-11-25,1364760,2022-11-25,300.087669,304.630323,293.529879,300.225753,9.101830e+06,9.063683e+06,23345606,23373804,1.0,16136.0,14800.0,NaN,1.816551e+07
2022-11-26,1364760,2022-11-26,300.225295,316.787367,300.139182,311.051139,1.072095e+07,9.386321e+06,23373806,23402018,1.0,15987.0,15941.0,NaN,2.010727e+07
2022-11-27,1364760,2022-11-27,311.050841,316.365007,306.025563,306.773434,9.036277e+06,9.531502e+06,23402024,23430502,1.0,15283.0,14602.0,NaN,1.856778e+07
2022-11-28,1364760,2022-11-28,306.773488,308.490684,289.239530,293.141829,1.015980e+07,1.179148e+07,23430504,23458977,1.0,17488.0,14359.0,NaN,2.195128e+07


## Draw candle chart

- Display the price chart using OHLCV candles
- We have a shortcut function for this, but you can as well construct Plotly
  chart by hand

In [19]:
from tradingstrategy.charting.candle_chart import visualise_ohclv

figure = visualise_ohclv(
    candles,
    chart_name=f"{pair.base_token_symbol} - {pair.quote_token_symbol} price chart",
    y_axis_name=f"$ {pair.base_token_symbol} price",
)

display(figure)

## Dark theme for the candle chart

- You can update Plotly figure settings to change its colour theme.
- [See Plotly default colour scheme and theming options](https://plotly.com/python/templates).


In [20]:
figure.update_layout(template="plotly_dark")

display(figure)

## Further reading

For more advanced charting, `trade-executor` Python library provides charts
that plot out a trading strategy performance (as opposite to vanilla price chart).